## **Limpieza de los archivos Excel de "Emergencias UNGRD"**
----

#### ***Importación de paquetes***

In [341]:
import pandas as pd

## **Procesamiento de los datos**
---


### ***Declarar función para realizar la limpieza***

In [342]:
def limpiar_datos_emergencias(filepath, end_row, sheet_name='REPORTE DE EMERGENCIAS', skip_rows=1):
    # Leer el archivo Excel
    df = pd.read_excel(filepath, sheet_name=sheet_name, skiprows=skip_rows)
    
    # Seleccionar filas y columnas relevantes
    df = df[0:end_row]  # Restar 1 y el valor de skip_rows para que incluya la fila deseada
    
    # Definir los posibles nombres de columnas
    posibles_columnas = {
        "DEPTO": "DEPARTAMENTO",
        "DEPARTAMENTO": "DEPARTAMENTO",
        "MUERTOS": "FALLECIDOS",
        "HERIDOS": "HERIDOS",
        "DESAPA.": "DESAPARECIDOS",
        "PERSONAS": "PERSONAS",
        "FAMILIAS": "FAMILIAS",
        "VIV.DESTRU.": "VIVIENDAS",
        "VIAS": "VIAS",
        "HECTAREAS": "HECTAREAS"
    }
    
    # Crear un diccionario para el renombramiento basado en las columnas presentes
    columnas_presentes = {col: posibles_columnas[col] for col in df.columns if col in posibles_columnas}
    
    # Seleccionar solo las columnas presentes
    columnas_a_seleccionar = ["FECHA", "EVENTO", "MUNICIPIO"] + list(columnas_presentes.keys())
    df = df[columnas_a_seleccionar]
    
    # Renombrar columnas
    df = df.rename(columns=columnas_presentes)
    
    # Columnas a limpiar y convertir
    columnas_a_convertir = ["FALLECIDOS", "HERIDOS", "DESAPARECIDOS", "PERSONAS", "FAMILIAS", "VIVIENDAS", "VIAS", "HECTAREAS"]

    # Reemplazar valores no numéricos por '0' y remover comas
    df[columnas_a_convertir] = df[columnas_a_convertir].replace(['-', ',', ' ', '\n', '  ', '                    ', '                                                                  '], '0').fillna('0').replace(',', '')
    
    # Verificar y limpiar valores incorrectos
    for col in columnas_a_convertir:
        for index, value in df[col].items():
            if isinstance(value, str) and not value.replace('.', '', 1).isdigit():
                print(f"Valor no numérico encontrado en columna '{col}', fila {index}: {value}")
                df.at[index, col] = '0'
            elif isinstance(value, pd.Timestamp):  # Detectar fechas que no deberían estar en columnas numéricas
                print(f"Fecha encontrada en columna '{col}', fila {index}: {value}")
                df.at[index, col] = '0'

    # Convertir columnas a float y luego a int
    for col in columnas_a_convertir:
        if col in df.columns:  # Verificar si la columna existe antes de intentar convertirla
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float).astype(int)
    
    # Convertir la columna de fechas y extraer el año y el mes
    df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
    df['AÑO'] = df['FECHA'].dt.year
    df['MES'] = df['FECHA'].dt.month
    
    # Reorganizar las columnas
    cols = ['AÑO', 'MES'] + [col for col in df.columns if col not in ['AÑO', 'MES']]
    df = df[cols]    
    
    return df

### ***Procesar CSVs***
---

#### ***2003***

In [343]:
df_2003 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2003.xls', 561)

In [344]:
# df_2003.head()
# df_2003.tail()
df_2003["EVENTO"].unique()

array(['INUNDACION', 'VENDAVAL', 'INCENDIO FORESTAL',
       'INCENDIO ESTRUCTURAL', 'CONTAMINACION', 'SISMO', 'DESLIZAMIENTO',
       'AVALANCHA', 'ACCIDENTE AEREO', 'ACCIDENTE', 'SEQUIA', 'VARIOS',
       'GRANIZADA', 'ANTROPICO', 'COLAPSO ESTRUCTURAL', 'ERUPCION ',
       'EXPLOSION'], dtype=object)

#### ***2004***

In [345]:
df_2004 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2004.xls', 774)

Valor no numérico encontrado en columna 'HECTAREAS', fila 425: 69,25


In [346]:
# df_2004.head()
df_2004.tail()
# df_2004["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
769,2004,12,2004-12-26,VENDAVAL,FILANDIA,QUINDIO,0,0,0,137,26,0,0,0
770,2004,12,2004-12-26,VENDAVAL,CALARCA,QUINDIO,0,0,0,60,11,0,0,0
771,2004,12,2004-12-29,INUNDACION,LA PLATA,HUILA,1,1,0,329,80,10,0,0
772,2004,12,2004-12-29,INUNDACION,LEIVA,NARIÑO,0,0,0,2208,450,0,0,0
773,2004,12,2004-12-29,INUNDACION,TESALIA,HUILA,0,0,0,180,36,0,0,0


#### ***2005***

In [347]:
df_2005 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2005.xls', 990)

In [348]:
# df_2005.head()
df_2005.tail()
# df_2005["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
985,2005,12,2005-12-23,INUNDACION,NEIVA,HUILA,0,0,0,100,20,20,0,0
986,2005,12,2005-12-26,INUNDACION,BALBOA,CAUCA,0,0,0,1592,491,0,0,203
987,2005,12,2005-12-26,DESLIZAMIENTO,FLORENCIA,CAUCA,0,0,0,0,0,0,1,0
988,2005,12,2005-12-27,VENDAVAL,ATRATO,CHOCO,0,0,0,4330,866,0,0,0
989,2005,12,2005-12-28,INUNDACION,UTICA,CUNDINAMARCA,0,0,0,0,0,0,0,0


#### ***2006***

In [349]:
df_2006 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2006.xls', 1053)

In [350]:
# df_2006.head()
# df_2006.tail()
df_2006["EVENTO"].unique()

array(['INUNDACION', 'DESLIZAMIENTO', 'VENDAVAL', 'AVALANCHA',
       'INCENDIO ESTRUCTURAL', 'EXPLOSION', 'INCENDIO FORESTAL',
       'EVENTO VOLCANICO', 'ACCIDENTE', 'COLAPSO ESTRUCTURAL',
       'TORMENTA ELECTRICA', 'EROSION', 'CONTAMINACION', 'MAREJADAS',
       'VARIOS', 'GRANIZADA', 'ERUPCION', 'TORNADO', 'SEQUIA'],
      dtype=object)

#### ***2007***

In [351]:
df_2007 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2007.xls', 1011)

In [352]:
# df_2007.head()
# df_2007.tail()
df_2007["EVENTO"].unique()

array(['INUNDACION', 'VENDAVAL', 'MAR DE LEVA', 'DESLIZAMIENTO',
       'ACCIDENTE', 'INCENDIO ESTRUCTURAL', 'EROSION',
       'COLAPSO ESTRUCTURAL', 'INCENDIO FORESTAL', 'EXPLOSION', 'SEQUIA',
       'CONTAMINACION', 'SISMO', 'ERUPCION', 'TORMENTA ELECTRICA',
       'AVALANCHA', 'VARIOS', 'GRANIZADA', 'HURACAN'], dtype=object)

#### ***2008***

In [353]:
df_2008 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2008.xls', 1752, skip_rows=2)

In [354]:
# df_2008.head()
# df_2008.tail()
# df_2008["EVENTO"].unique()

#### ***2009***

In [355]:
df_2009 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2009.xls', 1063)

C:\Users\marti\AppData\Local\Temp\ipykernel_14260\2400448329.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[columnas_a_convertir] = df[columnas_a_convertir].replace(['-', ',', ' ', '\n', '  ', '                    ', '                                                                  '], '0').fillna('0').replace(',', '')


In [356]:
# df_2009.head()
# df_2009.tail()
# df_2009["EVENTO"].unique()

#### ***2010***

In [357]:
df_2010 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2010.xls', 2445)

In [358]:
# df_2010.head()
df_2010.tail()
# df_2010["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
2440,2010,12,2010-12-31,INUNDACION,GUADALUPE,SANTANDER,0,0,0,475,95,0,0,0
2441,2010,12,2010-12-31,INUNDACION,JORDAN,SANTANDER,0,0,0,27,6,0,0,0
2442,2010,12,2010-12-31,INUNDACION,SUAITA,SANTANDER,0,0,0,1535,307,0,0,0
2443,2010,12,2010-12-31,INUNDACION,GUEPSA,SANTANDER,0,0,0,20,4,0,0,0
2444,2010,12,2010-12-31,INUNDACION,LERIDA,TOLIMA,0,0,0,240,48,0,0,0


#### ***2011***

In [359]:
df_2011 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2011.xls', 3200)

In [360]:
df_2011.head()
# df_2011.tail()
# df_2011["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
0,2009,9,2009-09-20,INCENDIO ESTRUCTURAL,MONTERIA,CORDOBA,0,0,0,0,0,0,0,0
1,2011,1,2011-01-01,INCENDIO ESTRUCTURAL,BELLO,ANTIOQUIA,0,0,0,24,6,6,0,0
2,2011,1,2011-01-01,INCENDIO ESTRUCTURAL,SAN ANDRES DE TUMACO,NARIÑO,2,0,0,440,88,44,0,0
3,2011,1,2011-01-02,INUNDACION,LOPEZ DE MICAY,CAUCA,0,0,1,1600,405,0,0,0
4,2011,1,2011-01-02,DESLIZAMIENTO,QUIBDO,CHOCO,0,0,0,35,7,0,0,0


#### ***2012***

In [361]:
df_2012 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2012.xls', 4044)

Valor no numérico encontrado en columna 'HECTAREAS', fila 327:               0.5
Valor no numérico encontrado en columna 'HECTAREAS', fila 2721: 0.7 
Valor no numérico encontrado en columna 'HECTAREAS', fila 2788: 0.5 


In [362]:
# Ajustar manualmente los valores decimales en la columna 'HECTAREAS'
valores_hectareas = {327: 0.5, 2721: 0.7, 2788: 0.5}
for index, value in valores_hectareas.items():
    df_2012.at[index, 'HECTAREAS'] = value

C:\Users\marti\AppData\Local\Temp\ipykernel_14260\280601477.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_2012.at[index, 'HECTAREAS'] = value


In [363]:
df_2012.head()
# df_2012.tail()
# df_2012["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
0,2012,1,2012-01-01,INCENDIO FORESTAL,ANGELOPOLIS,ANTIOQUIA,0,0,0,0,0,0,0,4.0
1,2012,1,2012-01-01,INCENDIO FORESTAL,BELLO,ANTIOQUIA,0,0,0,0,0,0,0,1.0
2,2012,1,2012-01-01,INCENDIO FORESTAL,BELLO,ANTIOQUIA,0,0,0,0,0,0,0,1.0
3,2012,1,2012-01-01,INCENDIO FORESTAL,CALDAS,ANTIOQUIA,0,0,0,0,0,0,0,1.0
4,2012,1,2012-01-01,INCENDIO FORESTAL,GAMEZA,BOYACA,0,0,0,0,0,0,0,3.0


#### ***2013***

In [364]:
df_2013 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2013.xls', 4019)

In [365]:
# df_2013.head()
# df_2013.tail()
# df_2013["EVENTO"].unique()

#### ***2014***

In [366]:
df_2014 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2014.xls', 3679, skip_rows=3)

In [367]:
# df_2014.head()
# df_2014.tail()
df_2014["EVENTO"].unique()

array(['INCENDIO FORESTAL', 'INCENDIO ESTRUCTURAL', 'ACCIDENTE',
       'EXPLOSION', 'DESLIZAMIENTO', 'SISMO', 'COLAPSO', 'VENDAVAL',
       'INUNDACION', 'ACCIDENTE DE TRANSITO', 'AVALANCHA',
       'CRECIENTE SUBITA', 'ACCIDENTE AEREO', 'CONTAMINACION', 'SEQUIA',
       'ACCIDENTE MINERO', 'EROSION', 'GRANIZADA', 'OTROS',
       'TORMENTA ELECTRICA', 'INCENDIO VEHICULAR', 'ACCIDENTE MARITIMO',
       'MAR DE LEVA'], dtype=object)

#### ***2015***

In [368]:
df_2015 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2015.xls', 3683, skip_rows=3)

In [369]:
# df_2015.head()
# df_2015.tail()
df_2015["EVENTO"].unique()

array(['INCENDIO FORESTAL', 'ACCIDENTE', 'INCENDIO ESTRUCTURAL',
       'ACCIDENTE DE TRANSITO', 'VENDAVAL', 'MAR DE LEVA',
       'DESLIZAMIENTO', 'COLAPSO', 'ACCIDENTE MINERO', 'INUNDACION',
       'TORMENTA ELECTRICA', 'SEQUIA', 'AVALANCHA', 'CONTAMINACION',
       'GRANIZADA', 'SISMO', 'EROSION', 'ACCIDENTE AEREO',
       'CRECIENTE SUBITA', 'ERUPCION', 'ACCIDENTE ', 'AVENIDA TORRENCIAL',
       'ACCIDENTE FLUVIAL', 'OTROS', 'SEQUIA ', 'FALLA GEOLOGICA',
       'AVALANCHA ', 'EMERGENCIA SOCIAL', 'EXPLOSION', 'VENDAVAL ',
       'ACCIDENTE DE TRANSITO '], dtype=object)

#### ***2016***

In [370]:
df_2016 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2016.xls', 3866, skip_rows=3)

In [371]:
# df_2016.head()
df_2016.tail()
# df_2016["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
3861,2016,12,2016-12-31,INUNDACION,PEREIRA,RISARALDA,0,0,0,40,8,1,0,0
3862,2016,12,2016-12-31,INCENDIO FORESTAL,CHITARAQUE,BOYACA,0,0,0,0,0,0,0,1
3863,2016,12,2016-12-31,INCENDIO ESTRUCTURAL,TADO,CHOCO,0,0,0,6,2,2,0,0
3864,2016,12,2016-12-31,INCENDIO ESTRUCTURAL,TIMBIQUI,CAUCA,1,0,0,5,1,1,0,0
3865,2016,12,2016-12-31,DESLIZAMIENTO,IQUIRA,HUILA,0,0,0,0,0,0,0,0


#### ***2017***

In [372]:
df_2017 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2017.xls', 3333, skip_rows=3)

Valor no numérico encontrado en columna 'HECTAREAS', fila 1164: 493,5
Valor no numérico encontrado en columna 'HECTAREAS', fila 1914: 627,07
Valor no numérico encontrado en columna 'HECTAREAS', fila 2287: .
Valor no numérico encontrado en columna 'HECTAREAS', fila 2469: 0,5
Valor no numérico encontrado en columna 'HECTAREAS', fila 2471: 0,02
Valor no numérico encontrado en columna 'HECTAREAS', fila 2613: 0,25
Valor no numérico encontrado en columna 'HECTAREAS', fila 2740: 0,1
Valor no numérico encontrado en columna 'HECTAREAS', fila 3265: 1,5


In [373]:
df_2017['HECTAREAS'] = df_2017['HECTAREAS'].replace(',', '.').replace('.', '0')

In [374]:
# df_2017.head()
df_2017.tail()
# df_2017["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
3328,2017,12,2017-12-31,INUNDACION,CHOCONTA,CUNDINAMARCA,0,0,0,15,3,0,0,0
3329,2017,12,2017-12-31,ACCIDENTE DE TRANSITO,CAQUEZA,CUNDINAMARCA,1,1,0,0,0,0,0,0
3330,2017,12,2017-12-31,DESLIZAMIENTO,MEDELLIN,ANTIOQUIA,0,0,0,0,0,0,1,0
3331,2017,12,2017-12-31,INCENDIO FORESTAL,AGUAZUL,CASANARE,0,0,0,0,0,0,0,60
3332,2017,12,2017-12-31,VENDAVAL,"BOGOTA, D.C.","BOGOTA, D.C.",0,0,0,0,4,0,0,0


#### ***2018***

In [375]:
df_2018 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2018.xls', 3489, skip_rows=3)

Valor no numérico encontrado en columna 'HERIDOS', fila 2609: |
Valor no numérico encontrado en columna 'HECTAREAS', fila 3205: 137,25
Valor no numérico encontrado en columna 'HECTAREAS', fila 3387: 0,1
Valor no numérico encontrado en columna 'HECTAREAS', fila 3388: 0,1


C:\Users\marti\AppData\Local\Temp\ipykernel_14260\2400448329.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[columnas_a_convertir] = df[columnas_a_convertir].replace(['-', ',', ' ', '\n', '  ', '                    ', '                                                                  '], '0').fillna('0').replace(',', '')


In [376]:
valores_hectareas = {
    3205: 137.25,
    3387: 0.1,
    3388: 0.1
}

for index, value in valores_hectareas.items():
    df_2012.at[index, 'HECTAREAS'] = float(value)

df_2012.at[2609, 'HERIDOS'] = 0

In [377]:
# df_2018.head()
df_2018.tail()
# df_2018["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
3484,2018.0,12.0,2018-12-30,INCENDIO FORESTAL,AGUA DE DIOS,CUNDINAMARCA,0,0,0,0,0,0,0,1
3485,2018.0,12.0,2018-12-31,INCENDIO FORESTAL,LA CRUZ,NARIÑO,0,0,0,0,0,0,0,1
3486,2018.0,12.0,2018-12-31,VENDAVAL,SAN PABLO,NARIÑO,0,0,0,5,1,0,0,0
3487,2018.0,12.0,2018-12-31,INCENDIO ESTRUCTURAL,EL TAMBO,NARIÑO,0,0,0,10,2,2,0,0
3488,2018.0,12.0,2018-12-31,INCENDIO FORESTAL,CASTILLA LA NUEVA,META,0,0,0,0,0,0,0,3


#### ***2023***

In [378]:
df_2023 = limpiar_datos_emergencias('../Excel (2003 - 2018, 2023)/EMERGENCIAS-2023.xls', 5154, skip_rows=4)

In [379]:
# df_2023.head()
df_2023.tail()
# df_2023["EVENTO"].unique()

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FAMILIAS,PERSONAS,FALLECIDOS,HERIDOS,DESAPARECIDOS,VIVIENDAS,VIAS,HECTAREAS
5149,2023,12,2023-12-31,ACCIDENTE TRANSPORTE TERRESTRE,MELGAR,TOLIMA,0,0,2,0,0,0,0,0
5150,2023,12,2023-12-28,MOVIMIENTO EN MASA,FLORENCIA,CAQUETA,0,0,0,0,0,0,1,0
5151,2023,12,2023-12-06,INCENDIO FORESTAL,NATAGA,HUILA,0,0,0,0,0,0,0,0
5152,2023,12,2023-12-06,INCENDIO FORESTAL,GARZON,HUILA,0,0,0,0,0,0,0,0
5153,2023,12,2023-12-06,DESABASTECIMIENTO DE AGUA,VELEZ,SANTANDER,0,0,0,0,0,0,0,0


### ***Fusionar datasets***
---

In [380]:
# Lista de nombres de los dataframes
dataframes = ['df_2003', 'df_2004', 'df_2005', 'df_2006', 'df_2007', 'df_2008', 'df_2009', 'df_2010', 'df_2011', 'df_2012', 'df_2013', 'df_2014', 'df_2015', 'df_2016', 'df_2017', 'df_2018', 'df_2023']

# Crear una lista vacía para almacenar los dataframes
dfs = []

# Iterar sobre los nombres de los dataframes y agregarlos a la lista
for df_name in dataframes:
    df = globals()[df_name]
    dfs.append(df)

# Concatenar los dataframes en uno solo
emergencias = pd.concat(dfs)

# Exportar el dataframe combinado como CSV
emergencias.to_csv('../EmergenciasCC(2003-2018, 2023)_RegionPacifico.csv', index=False)